In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

from tqdm import tqdm

# Upload metadata

In [7]:
md = pd.read_csv('../matrices/metadata.tsv', sep='\t', index_col=0)
md.head()

,PANGAEA sample id,Station.label,Layer,Layer2,polar,lower.size.fraction,upper.size.fraction,Event.date,Latitude,Longitude,...,Gradient.Surface.temp(SST),Fluorescence,Density,Depth.Min.O2,Depth.Max.O2,Mean Flux at 150m,FluxAttenuation,NPP 8d VGPM (mgC/m2/day),Ocean,Province
TSC_NAME,,,,,,,,,,,,,,,,,,,,,
TSC000,TARA_X000000368,TARA_004,DCM,EPI,non polar,0.22,1.6,2009-09-15 18:00,36.5533,-6.5669,...,0.85990,NaN,26.886600,NaN,NaN,NaN,NaN,NaN,[NAO] North Atlantic Ocean (MRGID:1912),NaN
TSC001,TARA_Y200000002,TARA_004,SRF,EPI,non polar,0.22,1.6,2009-09-15 11:30,36.5533,-6.5669,...,0.73994,NaN,25.909909,NaN,NaN,NaN,NaN,686.0860,[NAO] North Atlantic Ocean (MRGID:1912),B7
TSC002,TARA_A200000159,TARA_007,DCM,EPI,non polar,0.22,1.6,2009-09-23 16:08,37.0541,1.9478,...,1.03750,NaN,27.003675,NaN,NaN,NaN,NaN,354.2245,[MS] Mediterranean Sea (MRGID:1905),NaN
TSC003,TARA_A200000113,TARA_007,SRF,EPI,non polar,0.22,1.6,2009-09-23 12:50,37.0510,1.9378,...,1.03750,NaN,25.611150,NaN,NaN,NaN,NaN,354.2245,[MS] Mediterranean Sea (MRGID:1905),B7
TSC004,TARA_X000001036,TARA_009,DCM,EPI,non polar,0.22,1.6,2009-09-28 16:59,39.0609,5.9422,...,0.35542,NaN,27.816050,NaN,NaN,NaN,NaN,205.1220,[MS] Mediterranean Sea (MRGID:1905),NaN


# Bins of environmental variables

In [10]:
bins_temperature = [-np.inf, 10, 22, np.inf]
labels_temperature = ['<10', '10-22', '>22']
md['Temperature_binned'] = pd.cut(md['Temperature'], bins=bins_temperature, labels=labels_temperature)

bins_oxygen = [-np.inf, 185, 250, np.inf]
labels_oxygen = ['<185', '185-250', '>250']
md['Oxygen_binned'] = pd.cut(md['Oxygen'], bins=bins_oxygen, labels=labels_oxygen)

md['ChlorophyllA_binned'] = np.where(md['ChlorophyllA'] <= 0.28, '<=0.28', '>0.28')

md['Fluorescence_binned'] = np.where(md['Fluorescence'] <= 2.3, '<=2.3', '>2.3')

bins_salinity = [-np.inf, 34, 37, np.inf]
labels_salinity = ['<=34', '34-37', '>37']
md['Salinity_binned'] = pd.cut(md['Salinity'], bins=bins_salinity, labels=labels_salinity)

md['NO3_binned'] = np.where(md['NO3'] <= 7, '<=7', '>7')

bins_flux = [-np.inf, 0.7, 3, np.inf]
labels_flux = ['<=0.7', '0.7-3', '>3']
md['Mean_Flux_150m_binned'] = pd.cut(md['Mean Flux at 150m'], bins=bins_flux, labels=labels_flux)

bins_npp = [-np.inf, 275, 540, np.inf]
labels_npp = ['<=275', '275-540', '>540']
md['NPP_binned'] = pd.cut(md['NPP 8d VGPM (mgC/m2/day)'], bins=bins_npp, labels=labels_npp)


# CLR implementation

In [11]:
def clr_(data, eps=1e-6):
    """
    Perform centered log-ratio (clr) normalization on a dataset.

    Parameters:
    data (pandas.DataFrame): A DataFrame with samples as rows and components as columns.

    Returns:
    pandas.DataFrame: A clr-normalized DataFrame.
    """
    if (data < 0).any().any():
        raise ValueError("Data should be strictly positive for clr normalization.")

    # Add small amount to cells with a value of 0
    if (data <= 0).any().any():
        data = data.replace(0, eps)

    # Calculate the geometric mean of each row
    gm = np.exp(data.apply(np.log).mean(axis=1))

    # Perform clr transformation
    clr_data = data.apply(np.log).subtract(np.log(gm), axis=0)

    return clr_data

# Model

In [ ]:
matrices = ['Matrix_MX_all', 'Matrix_M1_all', 'Matrix_M0_all', 'Matrix_guidi_all', 'Matrix_salazar_all', 'Matrix_stress_all',
           'Matrix_GEN_M4_all', 'Matrix_GEN_M0_all', 'Matrix_GEN_guidi_all', 'Matrix_GEN_salazar_all', 'Matrix_GEN_stress_all'
           ]
variables = ['polar', 'Layer', 'Layer2', 'Province', 'Temperature_binned', 'Oxygen_binned', 'ChlorophyllA_binned', 'Fluorescence_binned', 'Salinity_binned', 'NO3_binned', 'Mean_Flux_150m_binned', 'NPP_binned']
#variables = ['Temperature_binned', 'Oxygen_binned']
num_cycles = 100  # Number of cycles to run

results = []

# Label encoder instance
label_encoder = LabelEncoder()

# Iterate over each matrix and variable to train the model
for matrix_name in tqdm(matrices, desc='Processing matrices'):
    matrix = pd.read_csv(f'../matrices/{matrix_name}.tsv', sep='\t', index_col=0)
    matrix = clr_(matrix)  # Apply clr transformation
    
    for variable in tqdm(variables, desc=f'Variables in {matrix_name}', leave=False):
        X = matrix
        y = label_encoder.fit_transform(md[variable])  # Encode labels
        
        # Store f1 scores for each cycle
        f1_scores = []

        for cycle in tqdm(range(num_cycles), desc='Simulation cycles', leave=False):
            # Split the data into training and testing sets with a different random state each time
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=cycle)

            # Create and train the model
            model = XGBClassifier(n_estimators=250)
            model.fit(X_train, y_train)

            # Evaluate the model
            y_pred = model.predict(X_test)
            score = f1_score(y_test, y_pred, average='weighted')
            f1_scores.append(score)

        # Compute average F1 score over all cycles
        avg_f1_score = np.mean(f1_scores)
        results.append(('_'.join(matrix_name.split('_')[1:]), variable, avg_f1_score))

# Filename
output_file = 'initial_prediction_tf_vs_gen'
# Directory
out_dir = '../../out_results/out_initial_predictions/'
        
df_results = pd.DataFrame(results, columns=['matrix_type', 'target_variable', 'f1_score'])
df_results.to_csv(out_dir+output_file+'.tsv', sep='\t', index=False)

# Display the results
for result in results:
    print(f'Matrix: {result[0]}, Variable: {result[1]}, Average F1 Score over {num_cycles} cycles: {result[2]:.4f}')


Variables in Matrix_MX_all:   0%|          | 0/12 [00:00<?, ?it/s]

Simulation cycles:   0%|          | 0/100 [00:00<?, ?it/s]

Simulation cycles:   2%|▏         | 2/100 [00:00<00:08, 11.12it/s]

Simulation cycles:   4%|▍         | 4/100 [00:00<00:07, 12.23it/s]

Simulation cycles:   6%|▌         | 6/100 [00:00<00:07, 12.48it/s]

Simulation cycles:   8%|▊         | 8/100 [00:00<00:07, 12.45it/s]

Simulation cycles:  10%|█         | 10/100 [00:00<00:06, 12.92it/s]

Simulation cycles:  12%|█▏        | 12/100 [00:00<00:06, 12.75it/s]

Simulation cycles:  14%|█▍        | 14/100 [00:01<00:06, 12.60it/s]

Simulation cycles:  16%|█▌        | 16/100 [00:01<00:06, 12.25it/s]

Simulation cycles:  18%|█▊        | 18/100 [00:01<00:07, 11.14it/s]

Simulation cycles:  20%|██        | 20/100 [00:01<00:07, 11.24it/s]

Simulation cycles:  22%|██▏       | 22/100 [00:01<00:06, 11.76it/s]

Simulation cycles:  24%|██▍       | 24/100 [00:02<00:06, 11.62it/s]

Simulation cycles:  26%|██▌       | 26/100 [00:0